# Kotlin programming

## Find sublist

An obvious solution:

In [1]:
fun pruneList(list: List<Int>, threshold: Int): List<Int> = list.filter { it >= threshold }

require(pruneList(listOf(1, 2, 3, 4, 5, 6), 5) == listOf(5, 6))

## Find smallest element

Also pretty obvious! We are still just ascertaining basic familiarty with code:

In [2]:
fun smallest(list: List<Int>): Int = list.min()

require(smallest(listOf(5, 3, 2, 4)) == 2)

## Find stats

This good-enough-for-now solution traverses the list four times:

In [3]:
import java.math.BigInteger
import kotlin.collections.fold

interface Stats {
    val min: Int
    val max: Int
    val sum: Long
    val mean: Double
}

fun stats(list: List<Int>): Stats = object : Stats {
    override val min: Int by lazy { list.min() }
    override val max get() = list.max()
    override val sum = list.sumOf { it.toLong() }
    override val mean = sum.toDouble() / list.size;
}

val stats = stats(listOf(1, 2, 3, 4, 5))

require(stats.min == 1)
require(stats.max == 5)
require(stats.mean == 3.0)
require(stats.sum == 15L)

## Find stats with provider


### Subtask: Test support

This `provider()` works by repeatedly invoking in iterator:

In [4]:
fun List<Int>.provider(): () -> Int? = iterator().run {
    { takeIf { hasNext() }?.let { next() } }
}

val provider = listOf(1, 2, 3).provider()

require(provider() == 1)
require(provider() == 2)
require(provider() == 3)
require(provider() == null)

### Main task

This solution works by building a `Stats` implementation during one pass of the list.  It computes a `mean` on-demand.

An alternative solution uses an imperative approach, using a `while` loop.

The Big Hint here is that the `() -> Int?` signature fits right into the `generateSequence` function which lets us traverse, and fold, the list with ease:

In [5]:
import java.math.BigInteger

interface Stats {
    val min: Int
    val max: Int
    val sum: BigInteger
    val mean: Double
}

fun stats(list: () -> Int?): Stats {
    data class StatsImpl(
        override val min: Int,
        override val max: Int,
        override val sum: BigInteger,
        val count: Int
    ) : Stats {

        constructor() : this(Int.MAX_VALUE, Int.MIN_VALUE, BigInteger.ZERO, 0)

        override val mean: Double
            get() = count.takeIf { it > 0 }
                ?.let { (sum.div(it.toBigInteger())).toDouble() }
                ?: 0.toDouble()

        operator fun plus(i: Int) = StatsImpl(
            min(this.min, i),
            max(this.max, i),
            this.sum + i.toBigInteger(),
            this.count + 1
        )
    }

    return generateSequence(list).fold(StatsImpl()) { stats, integer ->
        stats + integer
    }
}

fun statsImperative(list: () -> Int?): Stats {
    var next = list()
    var min = next!!
    var max = min
    var count = 0
    var sum = 0.toBigInteger()

    while (next != null) {
        min = minOf(min, next)
        max = maxOf(max, next)
        sum += next.toBigInteger()
        count++
        next = list()
    }

    return object : Stats {
        override val min = min
        override val max = max
        override val sum = sum
        override val mean = sum.toDouble() / count.toDouble()
    }
}

fun List<Int>.provider(): () -> Int? = iterator().run {
    { takeIf { hasNext() }?.let { next() } }
}

val provider = listOf(1, 2, 3, 4, 5).provider()

val stats = stats(provider)

require(stats.min == 1)
require(stats.max == 5)
require(stats.mean == 3.0)
require(stats.sum == 15.toBigInteger())

## Provide a hook

The generic function is really easy when we already have a folding-based implementation, shown below.

An alternative imperative solution is given with `processImperative`.

In [6]:
fun <S> process(list: () -> Int?, initial: () -> S, processor: (S, Int) -> S): S =
    generateSequence(list).fold(initial(), processor)

fun <S> processImperative(
    list: () -> Int?,
    initial: () -> S,
    processor: (S, Int) -> S
): S {
    var value = initial()
    var next = list()

    while (next != null) {
        value = processor(value, next)
        next = list()
    }
    return value
}

fun List<Int>.provider(): () -> Int? = iterator().run {
    { takeIf { hasNext() }?.let { next() } }
}

val initial = { 0 }
val processor: (Int, Int) -> Int = { i1, i2 -> i1 + i2 }
val provider = listOf(1, 2, 3, 4, 5).provider()

require(process(provider, initial, processor) == 15)
